In [90]:
import cv2 
from ultralytics import YOLO
from collections import defaultdict

#Load the yolo model
model = YOLO('yolo11l.pt')


In [91]:
class_list = model.names


In [92]:
cap = cv2.VideoCapture('test_videos/4.mp4')
line_y_red = 430
# dictionary to store object count by class
class_counts = defaultdict(int)

#dictionary to keep track of objects IDs that have crossed the line
crossed_ids = set()

In [ ]:
while cap.isOpened():
    ret, frame = cap.read() #reading the frames one by one
    if not ret:
        break

 # Run yolo tracking on the frames
    results = model.track(frame, persist=True, classes = [1,2,3,5,6,7])

    # Ensure results are not empty
    if results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        cv2.line(frame, (690, line_y_red), (1130, line_y_red), (0, 0, 255), 3)
        

 # Loop through each detected object
    for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
        x1, y1, x2, y2 = map(int, box)
        cx = (x1 + x2) // 2      #calculate the centre point
        cy = (y1 + y2) // 2
        

        class_name = class_list[class_idx]

        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
        cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 -10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)

        # check if the object has crossed the red line
        if cy > line_y_red and track_id not in crossed_ids:
            # mark the objects as crossed
            crossed_ids.add(track_id)
            class_counts[class_name] +=1

        # display the counts on the frame
        y_offset = 30
        for class_name, count in class_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            y_offset += 30
                        
 
    cv2.imshow("YOLO Object Tracking & Counting", frame)

 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

 


0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 260.6ms
Speed: 1.3ms preprocess, 260.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 244.7ms
Speed: 1.2ms preprocess, 244.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 208.5ms
Speed: 1.0ms preprocess, 208.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 211.6ms
Speed: 1.1ms preprocess, 211.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 4 motorcycles, 3 buss, 1 truck, 207.0ms
Speed: 1.2ms preprocess, 207.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 4 motorcycles, 3 buss, 1 truck, 199.4ms
Speed: 1.2ms preprocess, 199.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 4 motorcycles, 3 buss, 2 t